In [ ]:
"""
TO-DO:
"""

In [ ]:
# # Need to iterate through each file 
# for root, subdirectories, files in os.walk(currentPSD_dir):
#     for file in files:
#         # fileName, fileExtension = os.path.splitext(file)
#         fileName = os.path.join(currentPSD_dir, file)
#         print("Modifying file: {}".format(file))
        
#         # Receives pricing tab from Pricing PSD and returns openpyxl object to modify
#         theFile = openpyxl.load_workbook(fileName)
#         # allSheetNames = theFile.sheetnames
#         # priceSheet = theFile[allSheetNames[1]]
#         for sheet in theFile.worksheets:
#             if sheet.title in pricingTabs:
#                 priceSheet = sheet
                
#                 lastRow = findLastRow(priceSheet)    
                 
#                 # For each file, find which column contains PartNumber and ListPrice
#                 partNumCol = find_specific_cell("PartNumber", "col") 
#                 listPriceColumn = find_specific_cell("Price", "col")  
                                                
#                 # Iterates through each cell in spreadsheet to find matches from Big Book
#                 for row in range(1, lastRow):
#                     partNumCell = "{}{}".format(partNumCol, row)
#                     listPriceCell = "{}{}".format(listPriceColumn, row)
#                     for index, entry in updatedPricesPN.iterrows():
#                         # if type(entry['Part Number']) != int and type(entry['Part Number']) != str:
#                         #     print("pn: {}, type: {}".format(entry['Part Number'], type(entry['Part Number'])))
#                         if str(priceSheet[partNumCell].value) == entry['Part Number']:
#                             # print("part number match! {} == {}".format(priceSheet[partNumCell].value, entry['Part Number']))
#                             print("Changing part number {} from {} to {}".format(priceSheet[partNumCell].value, priceSheet[listPriceCell].value, entry['2022 List Prices']))
#                             priceSheet[listPriceCell] = entry['2022 List Prices']
                    
#                 theFile.save(fileName)      
    

### New Code

In [ ]:
# Pseudo Code
"""
- Convert original pricing PSD to dataframe
- Create dataframe or dictionary containing partnumber, SAP description, and new pricing data
- Iterate through and update pricing in dataframe referencing pricing dictionary/dataframe



- Add 2 columns: 1 to contain original pricing, and 1 to calculate percent difference?
- Write new tabs to excel
"""

#### Imports

In [75]:
import pandas as pd
import os
import openpyxl

#### Convert original pricing PSD to dataframe

In [87]:
# Folder where PSDs are located
fileDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\20-29 Areas\22 grundfos-express-tools\Price Updater\Input PSDs"

file_to_change = r"CR_StdPumps.xlsx"
filePath = os.path.join(fileDir, file_to_change)

# Create dataframe from PSD
psd_data = pd.read_excel(filePath, sheet_name=1, header=1)

# This removes the header data from the dataframe to avoid duplicating during excel write operation
psd_data = psd_data.iloc[3::,:] 

#### Create dataframe or dictionary containing partnumber, SAP description, and new pricing data

In [88]:
newDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\20-29 Areas\22 grundfos-express-tools\Price Updater"
newPrices = r"2022_Big_Book.xlsx"
pathNewPrices = os.path.join(newDir, newPrices)

# Create dataframe of new prices
cols = ['Part Number', '2022 List Prices']
newPrice_data = pd.read_excel(pathNewPrices, sheet_name=0, usecols=cols)

# Renaming part number column for use in pandas merge
newPrice_data.rename(columns = {'Part Number':'Big Book Part Number'}, inplace=True)

### Reorder df column function

In [47]:
def reorder_columns(dataframe, col_name, position):
    """Reorder a dataframe's column.
    Args:
        dataframe (pd.DataFrame): dataframe to use
        col_name (string): column name to move
        position (0-indexed position): where to relocate column to
    Returns:
        pd.DataFrame: re-assigned dataframe
    """
    temp_col = dataframe[col_name]
    dataframe = dataframe.drop(columns=[col_name])
    dataframe.insert(loc=position, column=col_name, value=temp_col)
    return dataframe

#### Iterate through and update pricing in dataframe referencing pricing dictionary/dataframe

In [89]:
# Essentially a VLOOKUP in python
df = psd_data.merge(newPrice_data, how='left', left_on='PartNumber', right_on='Big Book Part Number')

# Re ordering columns for QA review
# last_col = len(df3.columns)
# df = reorder_columns(dataframe=df3, col_name='Big Book Part Number', position=last_col)
# df = reorder_columns(dataframe=df, col_name='2022 List Prices', position=last_col)

### Create Working Copy of pricing PSD; leaving original untouched

In [90]:
from openpyxl import load_workbook
import shutil

filename, file_ext = file_to_change.split(os.extsep)

outputDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\20-29 Areas\22 grundfos-express-tools\Price Updater\Output PSDs"
outputFilename = filename + " - Ready for Review." + file_ext
outputPath = os.path.join(outputDir, outputFilename)

shutil.copyfile(filePath, outputPath) # Create copy to leave original untouched

'C:\\Users\\104092\\OneDrive - Grundfos\\Documents\\20-29 Areas\\22 grundfos-express-tools\\Price Updater\\Output PSDs\\CR_StdPumps - Ready for Review.xlsx'

### Write to new tab in excel sheet

In [91]:
# Write dataframe 
with pd.ExcelWriter(outputPath, mode="a", engine="openpyxl",if_sheet_exists="overlay") as writer:
    # df3.to_excel(writer, sheet_name="CP BpaQ", header=None, startrow=writer.sheets["Sheet1"].max_row, index=False)
    df.to_excel(writer, sheet_name="CR BpaQ", header=None, startrow=5, index=False)